# Experimentación modelo predictivo

## Imports

Primero, se importan las dependencias necesarias para la experimentación. Principalmente se trabajará con tensorflow, para calcular metricas, y keras para construir redes neuronales, instanciar redes ya existentes, crear generadores de datos, realizar data augmentatión.

In [1]:
import tensorflow as tf
from keras import regularizers, initializers, Model
from keras.applications import Xception, VGG16, VGG19, ResNet50, InceptionV3, InceptionResNetV2, MobileNet, DenseNet121
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Input, concatenate
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


## Parametros 

Luego, se definen los parametros para la carga de imagenes y el entrenamiento.

In [2]:
# Parameters for data loading
batch_size = 16
imgage_size = 224
train_directory = 'src/data/training'
test_directory = 'src/data/test'
train_images = 1618
test_images = 404
class_mode = 'binary'

# Training parameters
epochs = 100
save_as= 'Experiment_Xception_transfer' # Name of the files where weigths and history will be saved

## Definición de metricas

Para la experimentación se utlizará metricas de precision, recall, specificity y kappa. Estas metricas se definen a continuación para, posteriormente, agregarlas al entrenamiento del modelo.

In [3]:
# Custom metric for measuring precision
def precision(y_true, y_pred):
    return tf.metrics.precision(y_true,y_pred)
    #TP,FP = 0,0
    #print(len(y_pred))
    #if (y_pred == 'positive' and y_true==y_pred):
    #    TP +=1
    #elif (y_pred == 'positive' and y_true!=y_pred):
    #    FP +=1
    #return TP / (TP+FP)
        
# Custom metric for measuring recall    
def recall(y_true, y_pred):
    TP,FN = 0,0
    if (y_pred == 'positive' and y_true==y_pred):
        TP +=1
    elif (y_pred == 'negative' and y_true!=y_pred):
        FN +=1
    return TP / (TP+FN)

# Custom metric for measuring specificity
def specificity(y_true, y_pred):
    FP,TN = 0,0
    if (y_pred == 'positive' and y_true!=y_pred):
        FP +=1
    elif (y_pred == 'negative' and y_true==y_pred):
        TN +=1
    return TP / (TP+FP)

# Custom metric for measuring kappa
def kappa(y_true, y_pred):
    TP,FP,TN,FN = 0,0,0,0
    if (y_pred == 'positive' and y_true==y_pred):
        TP +=1
    elif (y_pred == 'positive' and y_true!=y_pred):
        FP +=1
    elif (y_pred == 'negative' and y_true==y_pred):
        TN +=1
    elif (y_pred == 'negative' and y_true!=y_pred):
        FN +=1
    TOTAL = (TP+TN+FP+FN)
    OA = (TP+TN)/TOTAL
    AC = ((TP+FP)/TOTAL)*((TP+FN)/TOTAL)+((FN+TN)/TOTAL)*((FP+TN)/TOTAL)
    return  (OA-AC)/(1-AC) 

## Definición de función de entrenamiento

A continuación se define la función con la que se entrenarán los diferentes modelos. Se compila con las metricas definidad anteriormente; se realiza data augmentatión y se entrena el modelo. La función para entrenar recibe el modelo que debe entrenar y retorna la historia del entrenamiento.

In [4]:
def train_model(model):
    # Data generator to transform and rescale training images
    # Output: 4 posible transforms over original image
    #         (original, horizontal rotation, vertical rotation, horizontal + vertical)
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True)
    # Data generator to rescale test images
    test_datagen = ImageDataGenerator(rescale=1./255)
    # Data flow training images
    train_flow = train_datagen.flow_from_directory(
        directory=train_directory,  
        target_size=(imgage_size, imgage_size),  
        batch_size=batch_size,
        class_mode=class_mode)
    # Data flow test images
    test_flow = test_datagen.flow_from_directory(
        directory=test_directory,
        target_size=(imgage_size, imgage_size),
        batch_size=batch_size,
        class_mode=class_mode)
    # Compile model
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['acc'])#precision,recall,specificity,kappa])
    # Create check point call back to store best validation weigths
    bestWeigthsPath='src/trainingWeigths/best_' + save_as+'.hdf5'
    checkpoint = ModelCheckpoint(bestWeigthsPath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    # Run experiment
    history = model.fit_generator(
        generator=train_flow,
        steps_per_epoch=train_images//batch_size,
        epochs=epochs,
        validation_data=test_flow,
        validation_steps=test_images//batch_size,
        callbacks=[checkpoint],
        verbose=1)
    return history

## Exportar modelo

Para la posterior integración del modelo con una herramienta movil se exportará el modelo en formato .hdf5. Anteriormente, creamos un checkpoint que cumple con esta función; sin embargo, solo se exportan los pesos del modelo y no el modelo como tal. Teniendo en cuenta lo anterior es necesario cargar los pesos de los mejores resultados de entrenamiento (CheckPoint), y luego exportar el modelo entero (modelo + pesos).

In [5]:
def export(model):
    # Load checkpoint weigths
    model.load_weights('src/trainingWeigths/bestWeigths' + guardar_como+ '.hdf5')
    # Remove file
    os.remove('src/trainingWeigths/bestWeigths' + guardar_como+ '.hdf5')
    # Create new file saving model and weigths
    model.save('src/trainingWeigths/bestWeigths' + guardar_como+ '.h5')

## Cración de modelo predictivo

Luego se define una red convolucional y se procede a entrenar el modelo

In [ ]:
# Load custom model
def build_CNN_model():
    #Input
    inputs = Input(shape=(imgage_size,imgage_size,3,))
    #BranchOne
    branchOne = Conv2D(filters=16,kernel_size=(3,3),activation='relu',
                        kernel_initializer=initializers.RandomNormal(stddev=0.1))(inputs)
    branchOne = MaxPooling2D(pool_size=(2,2))(branchOne)
    branchOne = Conv2D(filters=32,kernel_size=(3,3),activation='relu', 
                       kernel_initializer=initializers.RandomNormal(stddev=0.1))(branchOne)
    branchOne = MaxPooling2D(pool_size=(2,2))(branchOne)    
    branchOne = Conv2D(filters=64,kernel_size=(3,3),activation='relu', 
                       kernel_initializer=initializers.RandomNormal(stddev=0.1))(branchOne)
    branchOne = Flatten()(branchOne)
    #BranchTwo
    branchTwo = Conv2D(filters=16,kernel_size=(5,5),activation='relu',
                        kernel_initializer=initializers.RandomNormal(stddev=0.1))(inputs)
    branchTwo = MaxPooling2D(pool_size=(2,2))(branchTwo)    
    branchTwo = Conv2D(filters=32,kernel_size=(5,5),activation='relu',
                        kernel_initializer=initializers.RandomNormal(stddev=0.1))(branchTwo)
    branchTwo = MaxPooling2D(pool_size=(2,2))(branchTwo) 
    branchTwo = Conv2D(filters=64,kernel_size=(5,5),activation='relu',
                        kernel_initializer=initializers.RandomNormal(stddev=0.1))(branchTwo)
    branchTwo = Flatten()(branchTwo)
    #BranchThree
    branchThree = Conv2D(filters=16,kernel_size=(3,3),activation='relu',
                          kernel_initializer=initializers.RandomNormal(stddev=0.1))(inputs)
    branchThree = MaxPooling2D(pool_size=(2,2))(branchThree)     
    branchThree = Conv2D(filters=32,kernel_size=(5,5),activation='relu',
                             kernel_initializer=initializers.RandomNormal(stddev=0.1))(branchThree)
    branchThree = MaxPooling2D(pool_size=(2,2))(branchThree) 
    branchThree = Conv2D(filters=64,kernel_size=(7,7),activation='relu',
                             kernel_initializer=initializers.RandomNormal(stddev=0.1))(branchThree)
    branchThree = Flatten()(branchThree)
    #Concatenate branches
    concatenated = concatenate([branchOne,branchTwo,branchThree], axis=1)
    #Clasification layers
    clasification = Dropout(0.3)(concatenated)
    clasification = Dense(124,activation='relu',kernel_regularizer=regularizers.l2(0.01),
                           kernel_initializer=initializers.RandomNormal(stddev=0.01))(clasification)
    clasification = Dropout(0.3)(clasification)
    #Output
    out = Dense(1, activation='sigmoid')(clasification)
    # Compile Model
    model = Model(inputs=[inputs], outputs=[out])
    model.summary()
    return model


model = build_CNN_model()
history = train_model(model)
export(model)

## Transfer learning

In [6]:
# Builds model for transfer learning and fine tunning
def build_transfer_learning_model(conv_layers):
    # Freeze conv layers that are not going to be trained
    for layer in conv_layers.layers[:]:
        layer.trainable = False 
    # Print summary of the layers
    for layer in conv_layers.layers:
        print(layer, layer.trainable)    
    # Create sequential model
    model = Sequential()
    # add conv layers to model
    model.add(conv_layers)
    # Add clasification layers to model
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    return model
    

### Xception

La primera red neuronal con la cual se experimentará será Xception. 

In [ ]:
# Instanciamos Xception
conv_layers = Xception(weights='imagenet',include_top=False,input_shape=(imgage_size, imgage_size, 3))
# Construimos modelo para transfer learning
model = build_transfer_learning_model(conv_layers)
history = train_model(model)
export(model)

### VGG16

In [ ]:
conv_layers = VGG16(weights='imagenet',include_top=False,input_shape=(imgage_size, imgage_size, 3))
model = build_transfer_learning_model(conv_layers)
history = train_model(model)
export(model, history)

### VGG19

In [ ]:
conv_layers = VGG19(weights='imagenet',include_top=False,input_shape=(imgage_size, imgage_size, 3))
model = build_transfer_learning_model(conv_layers)
history = train_model(model)
export(model, history)

### ResNet50

In [ ]:
conv_layers = ResNet50(weights='imagenet',include_top=False,input_shape=(imgage_size, imgage_size, 3))
model = build_transfer_learning_model(conv_layers)
history = train_model(model)
export(model)

/home/icesi/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


53182464/94653016 [===============>..............] - ETA: 2:43

### InceptionV3

In [ ]:
conv_layers = InceptionV3(weights='imagenet',include_top=False,input_shape=(imgage_size, imgage_size, 3))
model = build_transfer_learning_model(conv_layers)
history = train_model(model)
export(model, history)

### InceptionResnetV2

In [ ]:
conv_layers = InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(imgage_size, imgage_size, 3))
model = build_transfer_learning_model(conv_layers)
history = train_model(model)
export(model, history)

### MobileNet

In [ ]:
conv_layers = MobileNet(weights='imagenet',include_top=False,input_shape=(imgage_size, imgage_size, 3))
model = build_transfer_learning_model(conv_layers)
history = train_model(model)
export(model, history)

### DenseNet121

In [ ]:
conv_layers = DenseNet121(weights='imagenet',include_top=False,input_shape=(imgage_size, imgage_size, 3))
model = build_transfer_learning_model(conv_layers)
history = train_model(model)
export(model, history)